In [7]:
from libs.alibaba.visitor import Visitor

from libs.json import JSON
from openpyxl import load_workbook

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support.select import Select
from bs4 import BeautifulSoup
from pyquery import PyQuery as pq
import html
import time
import re
import os
import traceback
import types
import pendulum

In [8]:
market = JSON.deserialize('.', 'storage', 'markets.json')['Eyelashes']
visitor = Visitor(market, headless=False)

. storage markets.json
.\storage\markets.json was deserialized!
E:\glitter\Eyelashes_config . product_list.json
E:\glitter\Eyelashes_config\.\product_list.json was deserialized!


In [3]:
# visitor.update()

In [43]:
def parse_tr(self, tr):
    visitor = {}
    date = tr.find_element_by_css_selector('td.td-checkbox input').get_attribute('statdate')
    visitor['id'] = tr.find_element_by_css_selector('td.td-checkbox input').get_attribute('visitorid')
    visitor['idx'] = tr.find_element_by_css_selector('td.td-checkbox input').get_attribute('visitoridx')
    visitor['date'] = date
    visitor['region'] = tr.find_element_by_css_selector('td.td-region span').get_attribute('title')
    
    span_mailable = tr.find_element_by_css_selector('td.td-operate span:first-child')
    visitor['mailable_status'] = span_mailable.get_attribute('class')
    visitor['mailable_hint'] = span_mailable.get_attribute('hint')
    span_mail_count = tr.find_elements_by_css_selector('td.td-operate span.send-count')
    if span_mail_count:
        visitor['mail_sent_count'] = span_mail_count[0].get_attribute('count')
    else:
        visitor['mail_sent_count'] = 0
    
    pv_span = tr.find_element_by_css_selector('td.td-pv span')
    visitor['pv'] = pv_span.text

    visitor['pv-detail'] = []
    if visitor['pv'] != '0':
        css_pv_detail = '#J-visitor-detail'
        css_pv_detail_close = '#J-vistor-detail-close'
        css_pv_detail_body = '#J-visitor-detail-tbl-tbody'
        css_pv_detail_pagination = '#J-pagination-visitor-detail'
        pv_span.click()
        while True:
            pv_detail_tbody = WebDriverWait(self.browser, 15).until(
                EC.visibility_of_element_located((By.CSS_SELECTOR, css_pv_detail_body)))

            for pv_tr in pv_detail_tbody.find_elements_by_css_selector('tr'):
                pv = {}
                pv['idx'] = pv_tr.find_element_by_css_selector('td.visitor-detail-index').text
                pv['page'] = pv_tr.find_element_by_css_selector('td.visitor-detail-page a').get_attribute('href')
                page_acts = pv_tr.find_element_by_css_selector('td.visitor-detail-page div').text
                if '已发起询盘' in page_acts:
                    pv['inquiried'] = True
                else:
                    pv['inquiried'] = False

                if '已发起TradeManager咨询' in page_acts:
                    pv['tm_inquiried'] = True
                else:
                    pv['tm_inquiried'] = False
                pv['stay'] = pv_tr.find_element_by_css_selector('td.visitor-detail-stay').text
                pv['time'] = pv_tr.find_element_by_css_selector('td.visitor-detail-time').text
                visitor['pv-detail'].append(pv)

            next_pv_detail_button = self.browser.find_elements_by_css_selector(css_pv_detail_pagination + ' .ui-pagination-active + a')
            if next_pv_detail_button:
                while True:
                    try:
                        ActionChains(self.browser).move_to_element(next_pv_detail_button[0]).perform()
                        next_pv_detail_button[0].click()
                        break
                    except WebDriverException as e:
                        if 'is not clickable at point' in str(e):
                            self.browser.implicitly_wait(0.5)
                            continue
                        else:
                            raise e
            else:
                break

        while True:
            try:
                self.browser.find_element_by_css_selector(css_pv_detail_close).click()
                break
            except WebDriverException as e:
                if 'is not clickable at point' in str(e):
                    self.browser.implicitly_wait(0.5)
                    continue
                else:
                    raise e

    visitor['stay'] = tr.find_element_by_css_selector('td.td-stay-duration').text

    kws_div = tr.find_elements_by_css_selector('td.td-search-keywords>div.search-keywords')
    if kws_div:
        kws_div = kws_div[0]
        kws_text = kws_div.get_attribute('data-text')
        visitor['keywords'] =  re.sub('</div><div>', ',', kws_text)[5:-6].split(',')
        visitor['search_keyword_indices'] = []
        for idx, div in enumerate(kws_div.find_elements_by_css_selector('div')):
            if div.find_elements_by_css_selector('span.search-keyword'):
                visitor['search_keyword_indices'].append(idx)
    else:
        visitor['keywords'] = []
        visitor['search_keyword_indices'] = []

    visitor['minisite-acts'] = []
    for div in tr.find_elements_by_css_selector('td.td-minisite-active span'):
        visitor['minisite-acts'].append(div.get_attribute('textContent'))
    visitor['website-acts'] = []
    for div in tr.find_elements_by_css_selector('td.td-website-active span'):
        visitor['website-acts'].append(div.get_attribute('textContent'))

    return visitor
visitor.parse_tr = types.MethodType(parse_tr, visitor)

In [67]:
def mail_to_current_page_visitors(self, remain):
    css_tbody = '#J-visitors-tbl-tbody'
    tbody = WebDriverWait(self.browser, 15).until(
        EC.visibility_of_element_located((By.CSS_SELECTOR, css_tbody)))

    count = 0
    for tr in tbody.find_elements_by_css_selector(css_tbody+' tr.J-visitors-table-tr'):
        if count >= remain:
            break
        ActionChains(self.browser).move_to_element(tr).perform()

        visitor = self.parse_tr(tr)
        
        if not self.is_mail_needed(visitor):
            continue
    
        rps = []
        for pv in visitor['pv-detail']:
            url = pv['page']
            result = re.search('(_|\/)(\d+)(\.html|\/)', url, re.IGNORECASE)
            if result:
                ali_id = result.group(2)
                category = self.products[ali_id]['category'].lower()
                for key in self.recommended:
                    if key in category:
                        for rp in self.recommended[key]:
                            if rp not in rps:
                                rps.append(rp)
                        break

        self.mail_to_visitor(tr, rps)
        count += 1

    return remain - count
visitor.mail_to_current_page_visitors = types.MethodType(mail_to_current_page_visitors, visitor)

In [64]:
def is_mail_needed(self, visitor):
    
    if int(visitor['mail_sent_count']) > 0:
        print('mail_sent_count > 0')
        return False

    if visitor['region'].lower() == 'china':
        print('region is in china')
        return False

    mini_site_acts = ', '.join(visitor['minisite-acts'])
    if '已发起询盘' in mini_site_acts:
        print('已发起询盘')
        return False
    if '已发起TradeManager咨询' in mini_site_acts:
        print('已发起TradeManager咨询')
        return False

    rkws = ['lash', 'fan', 'volume', 'exten', 'cils', 'wimpern', 'pesta']
    found = False
    for kw in visitor['keywords']:
        kw = kw.lower()
        for rkw in rkws:
            if rkw in kw:
                found = True
                break
        if found:
            break
    if not visitor['keywords']:
        found = True
    
    pv_count = int(visitor['pv'])
    stay = visitor['stay']
    if stay == '-':
        stay = 0
    else:
        stay = int(stay.split('s')[0])
    if pv_count < 2 and stay < 10 and not found:
        print('pv_count < 2 and stay < 10 and absence of realated keywords')
        return False
       
    now = pendulum.now()
    dt_1500 = pendulum.parse('15:00', tz='Asia/Shanghai')
    dt_1600 = pendulum.parse('16:00', tz='Asia/Shanghai')
    if now < dt_1500 or now > dt_1600:
        if pv_count < 2 and stay < 10 and found:
            print('(<15:00 or >16:00) and pv_count < 2 and stay < 10')
            return False
    return True
visitor.is_mail_needed = types.MethodType(is_mail_needed, visitor)

In [2]:
# visitors = visitor.deserialize()
# len(visitors)

In [9]:
visitor.load_url()

primary 打开网址：https://i.alibaba.com
primary 等待登陆页面加载 ... ...
primary 输入登录信息， 并登录
success 成功 登录 阿里巴巴 国际站！用户名：Jeremy


In [66]:
visitor.mail_to_current_page_visitors(20)

region is in china
{'id': 'D18120411', 'idx': '11', 'date': '2018-12-04', 'region': 'China', 'mailable_status': 'unmailable', 'mailable_hint': 'unmailable', 'mail_sent_count': 0, 'pv': '2', 'pv-detail': [{'idx': '1', 'page': 'http://glittereyelash.en.alibaba.com/company_profile.html?spm=a2700.supplier-normal.35.3.45d41bc0W9HpBx', 'inquiried': False, 'tm_inquiried': False, 'stay': '4s', 'time': '2018-12-04 18:29:25'}, {'idx': '2', 'page': 'http://glittereyelash.en.alibaba.com/contactinfo.html?spm=a2700.icbuShop.0.0.61ce594dYHnfT7', 'inquiried': False, 'tm_inquiried': False, 'stay': '10s', 'time': '2018-12-04 18:29:29'}], 'stay': '14s', 'keywords': ['qingdao', 'qingdao eyelashes', 'qingdao packing', 'eyelash'], 'search_keyword_indices': [3], 'minisite-acts': ['访问Company Profile', '访问Contacts'], 'website-acts': ['总浏览量331', '浏览126个供应商']}

(<15:00 or >16:00) and pv_count < 2 and stay < 10
{'id': 'D18120413', 'idx': '13', 'date': '2018-12-04', 'region': 'Australia', 'mailable_status': 'unmai

20

In [ ]:
%store sorted_kws